    Topic:        Project 1  
    Subject:      Campaign Placement for WomenTechWomenYes Canvassing  
    Date:         07/09/2018  
    Name:         Courtney  
    Worked with:  Tim, Brandon

In [55]:
import json, urllib.request
import pickle
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
geolocator = Nominatim()

In [56]:
# read json file from internet that has station lat/long linked to station code
website = 'https://raw.githubusercontent.com/tmrudick/mta-turnstiles/master/data/stations.json'
with urllib.request.urlopen(website) as url:
        data = json.loads(url.read().decode())

In [57]:
# get lat/long for each station id as a dataframe
# UNIT will be first column, LAT second, LONG third

geo_df = pd.DataFrame(columns=['UNIT', 'LAT', 'LONG','ZIPCODE'])

lat = 'latitude'
long = 'longitude'
station_list = []
    
for station in data.items():
    station_geo = []
    station_geo.append(station[0])
    if lat in station[1].keys() and long in station[1].keys():
        station_geo.append(station[1][lat])
        station_geo.append(station[1][long])
        location = geolocator.reverse(station_geo[1] + ',' + station_geo[2])
        print("Add postal code " + location.raw['address']['postcode'][0:5])
        station_geo.append(location.raw['address']['postcode'][0:5])
    else:
        station_geo.append(None)
        station_geo.append(None)
        station_geo.append(None)
    station_list.append(station_geo)
geo_df = pd.DataFrame(station_list, columns = ['UNIT', 'LAT', 'LONG','ZIPCODE'])

Add postal code 10004
Add postal code 10038
Add postal code 11208
Add postal code 11421
Add postal code 11208
Add postal code 11421
Add postal code 11418
Add postal code 11418
Add postal code 11418
Add postal code 10036
Add postal code 10036
Add postal code 10011
Add postal code 10011
Add postal code 10038
Add postal code 10022
Add postal code 10022
Add postal code 10022
Add postal code 11373
Add postal code 11432
Add postal code 10036
Add postal code 10019
Add postal code 10019
Add postal code 10019
Add postal code 11433
Add postal code 11433
Add postal code 10038
Add postal code 10038
Add postal code 10007
Add postal code 10007
Add postal code 10001
Add postal code 10036
Add postal code 10036
Add postal code 10027
Add postal code 10039
Add postal code 10040
Add postal code 10034
Add postal code 10034
Add postal code 10463


KeyboardInterrupt: 

In [30]:
geo_df.tail(20)

,UNIT,LAT,LONG,ZIPCODE
460,R470,40.5446,-74.164581,10312
461,R526,None,None,None
462,R532,None,None,None
463,R535,40.660476,-73.830301,11414
464,R536,40.643942,-73.782356,11434
465,R537,40.643942,-73.782356,11434
466,R538,40.774702,-73.871189,11356
467,R540,40.711638,-74.010708,10005
468,R541,40.748931,-73.988113,10001
469,R542,40.742933,-73.992909,10011


In [31]:
geo_df.fillna(value=np.nan, inplace=True)

In [54]:
geo_df.tail(20)

,UNIT,LAT,LONG,ZIPCODE


In [41]:
with open('cleaned_4hr_interval_mta.pickle','rb') as read_file:
    time_series_df = pickle.load(read_file)

In [42]:
time_series_df.shape

(2519805, 13)

In [43]:
len(set(time_series_df['UNIT']))

468

In [44]:
len(set(geo_df['UNIT']))

480

In [45]:
time_series_df = pd.merge(time_series_df, geo_df, on='UNIT', how='outer')

In [46]:
time_series_df.shape

(2519821, 16)

In [47]:
time_series_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,ENTRIES,EXITS,DATE_TIME,DATE,DESC,TIME_INTERVAL,ENTRY_DIFF,EXIT_DIFF,LAT,LONG,ZIPCODE
0,A002,R051,02-00-00,59 ST,NQR456W,6566470.0,2224053.0,2018-03-31 03:59:00,2018-03-31,REGULAR,04:00:00,7.0,3.0,40.762796,-73.967686,10022
1,A002,R051,02-00-00,59 ST,NQR456W,6566470.0,2224053.0,2018-03-31 07:59:00,2018-03-31,REGULAR,04:00:00,0.0,0.0,40.762796,-73.967686,10022
2,A002,R051,02-00-00,59 ST,NQR456W,6566470.0,2224055.0,2018-03-31 11:59:00,2018-03-31,REGULAR,04:00:00,0.0,2.0,40.762796,-73.967686,10022
3,A002,R051,02-00-00,59 ST,NQR456W,6566470.0,2224055.0,2018-03-31 15:59:00,2018-03-31,REGULAR,04:00:00,0.0,0.0,40.762796,-73.967686,10022
4,A002,R051,02-00-00,59 ST,NQR456W,6566470.0,2224055.0,2018-03-31 19:59:00,2018-03-31,REGULAR,04:00:00,0.0,0.0,40.762796,-73.967686,10022


In [48]:
with open('spatial_4hr_interval_mta.pickle', 'wb') as to_write:
    pickle.dump(time_series_df, to_write)